# EXTRACCIÓN DE DATOS:

Paletas de sombras, pintalabios y colorete en función de la paleta de colores asignadas, siendo estas:\
        -PRIMAVERA\
        -VERANO\
        -OTOÑO\
        -INVIERNO

**IMPORTACIONES**

In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from webdriver_manager.chrome import ChromeDriverManager
from joblib import Parallel, delayed
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time

Ahora, vamos a scrappear los productos de maquillaje con mejor rating que se ajusten a las paletas de colores\
de cada estación. Sacaremos nombre, precio y rating.

In [2]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)  

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/235325829.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


*Existen varios productos que podemos recomendar segun la colometría de las personas. En este caso, nos centraremos\
en sombras de ojos, pintalabios, colorete y corrector*\

Por otra parte, debemos tener en cuenta que hay varias tiendas donde podemos encontrar productos de maquillaje.\
Concretamente, en nuestro proyecto, los escogeremos de las siguientes:

1. **PRIMOR**
2. **SEPHORA**

# **1 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS, PINTALABIOS Y COLORETE DE PRIMOR**

In [3]:
URL = 'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20verano&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and'
driver.get(URL)

In [44]:
#Investigamos dónde podemos entrar para scrappear PRIMOR
filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=108.0.5359.98)
Stacktrace:
0   chromedriver                        0x000000010a699f38 chromedriver + 4910904
1   chromedriver                        0x000000010a619a03 chromedriver + 4385283
2   chromedriver                        0x000000010a25e747 chromedriver + 472903
3   chromedriver                        0x000000010a234ff5 chromedriver + 303093
4   chromedriver                        0x000000010a2cdb0f chromedriver + 928527
5   chromedriver                        0x000000010a2e3763 chromedriver + 1017699
6   chromedriver                        0x000000010a2c8ee3 chromedriver + 909027
7   chromedriver                        0x000000010a29330c chromedriver + 688908
8   chromedriver                        0x000000010a29488e chromedriver + 694414
9   chromedriver                        0x000000010a6671de chromedriver + 4702686
10  chromedriver   

In [35]:
#Paletas de sombras
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO'] 
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    driver.quit()
     
    return total

In [36]:
links_sombras_primor = ['https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20verano&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20oto%C3%B1al&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20invierno&filter%5Bcategories%5D%5B0%5D=Ojos%20lujo&filter%5Bcategories%5D%5B1%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B2%5D=Sombras%20de%20ojos%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
               'https://www.primor.eu/#/dfclassic/query=paletas%20sombras%20pastel&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B1%5D=Sombras%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [37]:
productos = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_sombras_primor)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/840644339.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/840644339.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/840644339.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/840644339.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.7min remaining:  1.7min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.8min finished


In [8]:
sombras_ver_prim = productos[0] 
sombras_oto_prim = productos[1] 
sombras_inv_prim = productos[2] 
sombras_pri_prim = productos[3] 
#productos_totales = pd.concat([productos[0], productos[1], productos[2], productos[3]], axis = 0)

In [9]:
ad1 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20krash%20rosas&filter%5Bcategories%5D%5B0%5D=Paletas%20de%20sombras%20de%20ojos%20y%20maquillaje&filter%5Bcategories%5D%5B1%5D=Sombras%20de%20ojos&filter%5Bbrand%5D%5B0%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad2 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20lilas&filter%5Bcategories%5D%5B0%5D=Maquillaje%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad3 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20negras&filter%5Bcategories%5D%5B0%5D=Maquillaje%20de%20ojos&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
ad4 = extraer('https://www.primor.eu/#/dfclassic/query=sombras%20naranjas&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/535601540.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/535601540.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/535601540.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/535601540.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [10]:
#Limpiamos los índices y añadimos lo que hemos sacado más

#PRIMAVERA
sombras_pri_prim = pd.concat([sombras_pri_prim, ad1], axis = 0)
sombras_pri_prim.drop_duplicates(inplace = True)
sombras_pri_prim['ID'] = [e for e in range(len(sombras_pri_prim))]
sombras_pri_prim.set_index('ID', drop = True, inplace = True)

#OTOÑO
sombras_oto_prim = pd.concat([sombras_oto_prim, ad2], axis = 0)
sombras_oto_prim.drop_duplicates(inplace = True)
sombras_oto_prim['ID'] = [e for e in range(len(sombras_oto_prim))]
sombras_oto_prim.set_index('ID', drop = True, inplace = True)

#INVIERNO
sombras_inv_prim = pd.concat([sombras_inv_prim, ad3], axis = 0)
sombras_inv_prim.drop_duplicates(inplace = True)
sombras_inv_prim['ID'] = [e for e in range(len(sombras_inv_prim))]
sombras_inv_prim.set_index('ID', drop = True, inplace = True)

#VERANO
sombras_ver_prim = pd.concat([sombras_ver_prim, ad4], axis = 0)
sombras_ver_prim.drop_duplicates(inplace = True)
sombras_ver_prim['ID'] = [e for e in range(len(sombras_ver_prim))]
sombras_ver_prim.set_index('ID', drop = True, inplace = True)

In [11]:
#Pintalabios
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'PINTALABIOS'
    
    driver.quit()
     
    return total

In [12]:
links_pintalabios_primor = ['https://www.primor.eu/#/dfclassic/query=pintalabios%20vivid%20&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20neutro&filter%5Bcategories%5D%5B0%5D=Barras%20de%20labios%20lujo&filter%5Bcategories%5D%5B1%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20oscuro&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                           'https://www.primor.eu/#/dfclassic/query=pintalabios%20rosados&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&filter%5Bbrand%5D%5B0%5D=Revolution&filter%5Bbrand%5D%5B1%5D=NYX%20Professional%20Makeup&filter%5Bbrand%5D%5B2%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [13]:
productos1 = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_pintalabios_primor)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:  1.5min remaining:  1.5min
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.5min finished


In [14]:
#Añadimos más filas
add1 = extraer('https://www.primor.eu/#/dfclassic/query=pintalabios%20rosados&filter%5Bcategories%5D%5B0%5D=Pintalabios%20y%20maquillaje%20de%20labios&filter%5Bbrand%5D%5B0%5D=NYX%20Professional%20Makeup&filter%5Bbrand%5D%5B1%5D=Krash%20Kosmetics&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
add2 = extraer('https://www.primor.eu/#/dfclassic/query=berry%20nars&filter%5Bbrand%5D%5B0%5D=NARS&filter%5Bcategories%5D%5B0%5D=Labios%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_or')
add3 = extraer('https://www.primor.eu/#/dfclassic/query=nude%20nars&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')
add4 = extraer('https://www.primor.eu/#/dfclassic/query=dark%20nars&filter%5Bbrand%5D%5B0%5D=NARS&filter%5Bcategories%5D%5B0%5D=Barras%20de%20labios%20lujo&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_or')
add5 = extraer('https://www.primor.eu/#/dfclassic/query=mac%20lips&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and')

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1683671410.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver 

In [15]:
pintalab_ver_prim = productos1[0] 
pintalab_oto_prim = productos1[1] 
pintalab_inv_prim = productos1[2] 
pintalab_pri_pri = productos1[3]

In [17]:
#Limpiamos los índices y añadimos lo que hemos sacado más

#PRIMAVERA
pintalab_pri_prim = pd.concat([pintalab_pri_pri, add1, add3], axis = 0)
pintalab_pri_prim.drop_duplicates(inplace = True)
pintalab_pri_prim['ID'] = [e for e in range(len(pintalab_pri_prim))]
pintalab_pri_prim.set_index('ID', drop = True, inplace = True)

#OTOÑO
pintalab_oto_prim = pd.concat([pintalab_pri_pri, add1], axis = 0)
pintalab_oto_prim.drop_duplicates(inplace = True)
pintalab_oto_prim['ID'] = [e for e in range(len(pintalab_oto_prim))]
pintalab_oto_prim.set_index('ID', drop = True, inplace = True)

#INVIERNO
pintalab_inv_prim = pd.concat([pintalab_pri_pri, add4], axis = 0)
pintalab_inv_prim.drop_duplicates(inplace = True)
pintalab_inv_prim['ID'] = [e for e in range(len(pintalab_inv_prim))]
pintalab_inv_prim.set_index('ID', drop = True, inplace = True)

#VERANO
pintalab_ver_prim = pd.concat([pintalab_pri_pri, add5], axis = 0)
pintalab_ver_prim.drop_duplicates(inplace = True)
pintalab_ver_prim['ID'] = [e for e in range(len(pintalab_ver_prim))]
pintalab_ver_prim.set_index('ID', drop = True, inplace = True)

In [18]:
#Colorete
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 200);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'df-card__content')
    link = driver.find_elements(By.CLASS_NAME, 'df-card__main')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'COLORETE'
    
    driver.quit()
     
    return total

In [19]:
links_blush_primor = ['https://www.primor.eu/3ina/66421-colorete-the-blush.html#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=MAC&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=NARS&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush%20rosa&filter%5Bbrand%5D%5B0%5D=3ina&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and',\
                     'https://www.primor.eu/#/dfclassic/query=blush&filter%5Bbrand%5D%5B0%5D=Yves%20Saint%20Laurent&session_id=e68b0b5ac1828fdb430ff5b492adb6e5&query_name=match_and']

In [20]:
productos2 = Parallel(n_jobs=4, verbose=True)(delayed(extraer)(e) for e in links_blush_primor)

[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/2199440826.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/2199440826.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/2199440826.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/2199440826.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
[Parallel(n_jobs=4)]: Done   2 out of   4 | elapsed:   54.9s remaining:   54.9s
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:  1.1min finished


In [21]:
blush_pri_prim = productos1[0] 
blush_ver_prim = productos1[1] 
blush_inv_prim = productos1[2] 
blush_oto_prim = productos1[3]

# **2 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS Y COLORETES DE URBAN DECAY**

In [22]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)  

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/235325829.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [23]:
URL = 'https://www.urbandecay.com/eyeshadow/?prefn1=taxonomyColorEye&prefv1=BLACK%7CBLUE%7CBOLD%7CBROWN%7CCOOL%7CNEUTRALS%7CPLUMS%7CPURPLE%7CRED%7CSIENNA%7CSILVER%7CSMOKEY%7CTAUPE&start=0&sz=12#view-mode=grid'
driver.get(URL)

In [24]:
#Investigamos dónde podemos scrappear los datos que necesitamos
prueba = driver.find_elements(By.CLASS_NAME, 'c-product-tile__caption-inner')
nombres = driver.find_element(By.CLASS_NAME, 'c-product-tile__name').text
links1 = driver.find_elements(By.CLASS_NAME, 'c-product-image__link')
links = [e.get_attribute('href') for e in links1]
precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price') 
precios = [e.text.split()[-1] for e in precio]


In [28]:
#SOMBBRAS
def extraer(URL):
    PATH=ChromeDriverManager().install()
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, 'body>div.c-modal.h-text-align-center.m-borderless.m-autosize-large.m-image.m-opened.h-fade-in>div>button').click() 
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    nombre = driver.find_elements(By.CLASS_NAME, 'c-product-tile__name') #nombre
    time.sleep(5)
    precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price') #precio
    time.sleep(5)
    links1 = driver.find_elements(By.CLASS_NAME, 'c-product-image__link') #links
        
    tmp = [e.text for e in nombre]
    
    precios = [e.text.split('\n')[-1] for e in precio]
    
    links = [e.get_attribute('href') for e in links1]
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    driver.quit()
    
    return total

In [29]:
URL = 'https://www.urbandecay.com/naked-palettes/?start=0&sz=18'

In [30]:
sombras_ud = extraer(URL)

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1508465150.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [31]:
sombras_ud.drop_duplicates(inplace = True)
sombras_ud = sombras_ud[0:14] 

In [32]:
#Añadimos los valores de las paletas en función de los colores de las sombras

sombras_ud['PALETA'] = ['PRIMAVERA', 'PRIMAVERA / OTOÑO', 'OTOÑO / INVIERNO', 'VERANO', 'TODAS', 'PRIMAVERA / VERANO',\
                  'OTOÑO', 'PRIMAVERA / VERANO', 'OTOÑO / INVIERNO', 'PRIMAVERA', 'PRIMAVERA / INVIERNO', 'PRIMAVERA',\
                  'PRIMAVERA / OTOÑO / INVIERNO', 'OTOÑO / INVIERNO']

In [38]:
#COLORETE
def extraer(URL):
    PATH=ChromeDriverManager().install()
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    time.sleep(5)
    driver.find_element(By.CSS_SELECTOR, 'body>div.c-modal.h-text-align-center.m-borderless.m-autosize-large.m-image.m-opened.h-fade-in>div>button').click() 
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 3000);')  # 200px de scroll
    time.sleep(5)
    nombre = driver.find_elements(By.CLASS_NAME, 'c-product-tile__name') #nombre
    time.sleep(5)
    precio = driver.find_elements(By.CLASS_NAME, 'c-product-tile__price') #precio
    time.sleep(5)
    links1 = driver.find_elements(By.CLASS_NAME, 'c-product-image__link') #links
        
    tmp = [e.text for e in nombre]
    
    precios = [e.text.split('\n')[-1] for e in precio]
    
    links = [e.get_attribute('href') for e in links1]
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'COLORETE'
    
    driver.quit()
    
    return total

In [39]:
blush_ud = extraer('https://www.urbandecay.com/blush-bronzer-highlighter/')

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/169910402.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [40]:
blush_ud 

,NOMBRE,PRECIO,LINKS,PRODUCTO
0,"STAY NAKED THREESOME BRONZE, BLUSH, AND HIGHLI...",$37.00,https://www.urbandecay.com/threesome-by-urban-...,COLORETE
1,BEACHED BRONZER,$31.00,https://www.urbandecay.com/beached-bronzer-by-...,COLORETE
2,NAKED3 EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-3-palette-by-...,COLORETE
3,24/7 GLIDE-ON WATERPROOF EYELINER PENCIL,$11.50,https://www.urbandecay.com/24-7-glide-on-eye-p...,COLORETE
4,NAKED RELOADED EYESHADOW PALETTE,$22.00,https://www.urbandecay.com/naked-reloaded-urba...,COLORETE
5,NAKED HEAT EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-heat-eyeshado...,COLORETE
6,,,https://www.urbandecay.com/naked-honey-eyeshad...,COLORETE
7,,,https://www.urbandecay.com/urban-decay-cosmeti...,COLORETE
8,,,https://www.urbandecay.com/all-nighter-setting...,COLORETE
9,,,https://www.urbandecay.com/eyeshadow-primer-po...,COLORETE


In [41]:
blush_ud = blush_ud[0:2]
blush_ud.drop_duplicates(inplace = True)
blush_ud['PALETA'] = ['VERANO / INVIERNO', 'TODAS']


/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/3659531634.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blush_ud.drop_duplicates(inplace = True)
/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/3659531634.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  blush_ud['PALETA'] = ['VERANO / INVIERNO', 'TODAS']


In [42]:
mu_ud = pd.concat([sombras_ud, blush_ud], axis = 0)

In [43]:
mu_ud

,NOMBRE,PRECIO,LINKS,PRODUCTO,PALETA
0,NAKED3 EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-3-palette-by-...,SOMBRAS DE OJOS,PRIMAVERA
1,NAKED RELOADED EYESHADOW PALETTE,$22.00,https://www.urbandecay.com/naked-reloaded-urba...,SOMBRAS DE OJOS,PRIMAVERA / OTOÑO
2,NAKED HEAT EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/naked-heat-eyeshado...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
3,NAKED HONEY EYESHADOW PALETTE,$24.50,https://www.urbandecay.com/naked-honey-eyeshad...,SOMBRAS DE OJOS,VERANO
4,LET'S GET NAKED SET,$175.00,https://www.urbandecay.com/lets-get-naked-set-...,SOMBRAS DE OJOS,TODAS
5,UD X ROBIN EISENBERG ARTIST COLLAB HOLIDAY MAK...,$49.00,https://www.urbandecay.com/robin-eisenberg-hol...,SOMBRAS DE OJOS,PRIMAVERA / VERANO
6,NAKED WILD WEST MINI EYESHADOW PALETTE,$18.90,https://www.urbandecay.com/naked-wild-west-min...,SOMBRAS DE OJOS,OTOÑO
7,NAKED X ROBIN EISENBERG EYESHADOW PALETTE,$27.00,https://www.urbandecay.com/robin-eisenberg-nak...,SOMBRAS DE OJOS,PRIMAVERA / VERANO
8,NAKED WILD WEST EYESHADOW PALETTE,$24.50,https://www.urbandecay.com/urban-decay-cosmeti...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
9,NAKED3 MINI EYESHADOW PALETTE,$29.00,https://www.urbandecay.com/naked3-mini-eyeshad...,SOMBRAS DE OJOS,PRIMAVERA


# **3 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS Y COLORETES DE HUDA BEAUTY**

In [45]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH) 

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/91254812.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)


In [46]:
URL = 'https://hudabeauty.com/us/en_US/eyes/eyeshadow'
driver.get(URL)

In [50]:
#Investigamos cómo scrappear los datos de la web de HUDA BEAUTY
nombres = driver.find_element(By.CLASS_NAME, 'tile-body').text
links = driver.find_elements(By.CLASS_NAME, 'tile-body')

'Haze Obsessions Eyeshadow Palette\nPurple\nADD TO BAG\n$29.00\n$17.40'

In [68]:
#SOMBRAS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'tile-body')
    link = driver.find_elements(By.CLASS_NAME, 'link')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'COLOR', 'PRECIO ANT', 'PRECIO']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    total.drop(['PRECIO ANT'], axis = 1, inplace = True)
    
    
    driver.quit()
     
    return total

In [69]:
sombras_hb = extraer(URL)

/var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/514715876.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)


In [70]:
sombras_hb['PALETAS']=  ['PRIMAVERA / OTOÑO / INVIERNO', 'OTOÑO / INVIERNO', 'PRIMAVERA', 'OTOÑO / INVIERNO',\
                        'PRIMAVERA / OTOÑO', 'PRIMAVERA / OTOÑO / VERANO', 'VERANO / OTOÑO / INVIERNO',\
                        'PRIMAVERA / VERANO', 'OTOÑO / INVIERNO', 'TODAS', 'VERANO', 'PRIMAVERA / OTOÑO', \
                        'PRIMAVERA / OTOÑO', 'PRIMAVERA', 'OTOÑO / INVIERNO', 'OTOÑO / INVIERNO', 'VERANO / OTOÑO',\
                        'OTOÑO / PRIMAVERA', 'TODAS', 'OTOÑO / VERANO', 'VERANO']

In [71]:
sombras_hb #limpiar color y precio

,NOMBRE,TIPO,COLOR,PRECIO,LINKS,PRODUCTO,PALETAS
0,Haze Obsessions Eyeshadow Palette,Purple,ADD TO BAG,$17.40,https://hudabeauty.com/us/en_US/60-off/haze-ob...,SOMBRAS DE OJOS,PRIMAVERA / OTOÑO / INVIERNO
1,Wild Obsessions Eyeshadow Palette,Tiger,ADD TO BAG,None,https://hudabeauty.com/us/en_US/eyeshadow/wild...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
2,NUDE Obsessions Eyeshadow Palette,Light,ADD TO BAG,None,https://hudabeauty.com/us/en_US/eyeshadow/nude...,SOMBRAS DE OJOS,PRIMAVERA
3,NUDE Obsessions Eyeshadow Palette,Rich,ADD TO BAG,None,https://hudabeauty.com/us/en_US/eyeshadow/nude...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
4,Haze Obsessions Eyeshadow Palette,Khaki,ADD TO BAG,$17.40,https://hudabeauty.com/us/en_US/60-off/haze-ob...,SOMBRAS DE OJOS,PRIMAVERA / OTOÑO
5,Mini Matte & Metal Melted Shadows,Warm Browns,ADD TO BAG,$13.80,https://hudabeauty.com/us/en_US/sale/mini-melt...,SOMBRAS DE OJOS,PRIMAVERA / OTOÑO / VERANO
6,Wild Obsessions Eyeshadow Palette,Chameleon,ADD TO BAG,None,https://hudabeauty.com/us/en_US/eyeshadow/wild...,SOMBRAS DE OJOS,VERANO / OTOÑO / INVIERNO
7,Mini Matte & Metal Melted Shadows,Soft Pinks,ADD TO BAG,$13.80,https://hudabeauty.com/us/en_US/sale/mini-melt...,SOMBRAS DE OJOS,PRIMAVERA / VERANO
8,Brown Obsessions Eyeshadow Palettes,Caramel,ADD TO BAG,None,https://hudabeauty.com/us/en_US/eyeshadow/cara...,SOMBRAS DE OJOS,OTOÑO / INVIERNO
9,Matte Basics Melted Shadows,Black & White,ADD TO BAG,None,https://hudabeauty.com/us/en_US/cyber-week-sal...,SOMBRAS DE OJOS,TODAS


In [77]:
#LABIOS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'tile-body')
    link = driver.find_elements(By.CLASS_NAME, 'link')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'COLOR', 'PRECIO ANT', 'PRECIO', 'x']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    total.drop(['PRECIO ANT'], axis = 1, inplace = True)
    
    
    driver.quit()
     
    return total

In [78]:
URL = 'https://hudabeauty.com/us/en_US/makeup/lips'

In [81]:
pintalabios_hb = extraer(URL)

2022-12-10 15:29:51,252 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1072741941.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [83]:
#BLUSH
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    filas = driver.find_elements(By.CLASS_NAME, 'tile-body')
    link = driver.find_elements(By.CLASS_NAME, 'link')
    tmp = []
    links = []

    for e in filas:
        tmp.append(e.text.split('\n'))
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE', 'TIPO', 'COLOR', 'PRECIO ANT', 'PRECIO', 'x']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    total.drop(['PRECIO ANT'], axis = 1, inplace = True)
    
    
    driver.quit()
     
    return total

In [84]:
URL = 'https://hudabeauty.com/us/en_US/cheek/blush'

In [86]:
labios_ud = extraer(URL)

2022-12-10 15:31:14,947 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/670635853.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [87]:
labios_ud

,NOMBRE,TIPO,COLOR,PRECIO,x,LINKS,PRODUCTO
0,GloWish Cheeky Vegan Blush Powder,06 Sassy Saffron,ADD TO BAG,None,None,https://hudabeauty.com/us/en_US/cyber-week-sal...,SOMBRAS DE OJOS
1,Cheeky Tint Blush Stick,Perky Peach,ADD TO BAG,None,None,https://hudabeauty.com/us/en_US/cyber-week-sal...,SOMBRAS DE OJOS
2,Lovefest Cream Blush,Burning Cherry,ADD TO BAG,None,None,https://hudabeauty.com/us/en_US/cyber-week-sal...,SOMBRAS DE OJOS
3,ONLINE EXCLUSIVE,N.Y.M.P.H. KISS Glow Blush,40% SAVINGS,$39.00,$23.40,https://hudabeauty.com/us/en_US/blush/n.y.m.p....,SOMBRAS DE OJOS


# **4 - EXTRACCIÓN DE DATOS DE PRIMOR DE SOMBRAS Y COLORETES DE KIKO MILANO**

In [255]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH) 

2022-12-10 18:59:59,969 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/91254812.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [256]:
URL = 'https://www.kikocosmetics.com/es-es/maquillaje/labios.html'

In [257]:
driver.get(URL)

In [206]:
#Investigamos cómo scrappear la página de KIKO MILANO para poder sacar los productos y los precios
nombres = driver.find_elements(By.CLASS_NAME, 'ProductBox__Title')
precio = driver.find_elements(By.CLASS_NAME, 'js-new-price')
link2 =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Palette')

In [247]:
#BLUSH Y SOMBRAS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    nombre = driver.find_elements(By.CLASS_NAME, 'ProductBox__Title')
    precio = driver.find_elements(By.CLASS_NAME, 'js-new-price')
    link =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Palette')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    precios.remove('')
    precios.remove('')
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [248]:
URL = 'https://www.kikocosmetics.com/es-es/maquillaje/paletas/paleta-de-ojos.html'

In [249]:
sombras_kiko = extraer(URL) #meter paletas

2022-12-10 18:56:03,234 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/744290316.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



,NOMBRE,PRECIO,LINKS,PRODUCTO
0,Blue Me Maxi Eyeshadow Palette,"24,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
1,Joyful Holiday Shining Night Eyeshadow Palette,"24,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
2,A Holiday Fable Celestial Eyeshadow Palette,"24,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
3,New Cult Colours Eyeshadow Palette,"19,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
4,Cult Colours Eyeshadow Palette,"18,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
5,Smart Cult Eyeshadow Palette,"17,49 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
6,Holiday Gems Gorgeous Eyeshadow Palette,"16,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
7,A Holiday Fable Ethereal Eyeshadow Palette,"16,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
8,New Soft Nude Eyeshadow Palette,"16,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
9,Joyful Holiday Bright Dream Eyeshadow Palette,"16,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS


In [307]:
#PINTALABIOS
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    nombre = driver.find_elements(By.CLASS_NAME, 'ProductBox__Title')
    precio = driver.find_elements(By.CLASS_NAME, 'js-new-price')
    link =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Lip')
    link2 =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Unlimited')
    link3 =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Stick')
    link4 = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Pearly')
    link5 = driver.find_elements(By.PARTIAL_LINK_TEXT, 'Jelly')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    for e in link2:
        links.append(e.get_attribute('href'))
    
    for e in link3:
        links.append(e.get_attribute('href'))
    
    for e in link4:
        links.append(e.get_attribute('href'))
    
    for e in link5:
        links.append(e.get_attribute('href'))
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)
    
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    precios.remove('')
    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [308]:
URL = 'https://www.kikocosmetics.com/es-es/maquillaje/labios.html'

In [309]:
labios_kiko = extraer(URL)

2022-12-10 19:14:36,784 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/2374793197.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)



In [310]:
labios_kiko

,NOMBRE,PRECIO,LINKS,PRODUCTO
0,A Holiday Fable Enchanting Lipstick,"12,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
1,Joyful Holiday Sparkling Lips Lipstick,"12,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
2,Holiday Gems Diamond Dust Lipstick,"11,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
3,Joyful Holiday Matte Desire Lipstick,"11,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
4,Lost In Amalfi Glide&Bright Lip Stylo,"10,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
5,Green Me Lips & Cheeks,"10,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
6,Blue Me 3d Effect Lipstick Duo,"10,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
7,Blossoming Beauty 3-In-1 All Over Stick,"10,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
8,Joyful Holiday Hydra Desire Lipstylo,"10,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS
9,Unlimited Stylo,"9,99 €",https://www.kikocosmetics.com/es-es/maquillaje...,SOMBRAS DE OJOS


# 5 - EXTRACCIÓN DE DATOS DE PINTALABIOS DE KAT VON D

In [352]:
PATH=ChromeDriverManager().install()

driver=webdriver.Chrome(PATH)

2022-12-10 19:50:51,735 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/1981730666.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(PATH)



In [353]:
URL = 'https://kvdveganbeauty.com/collections/lip'

In [354]:
driver.get(URL)

In [350]:
nombres = driver.find_elements(By.CLASS_NAME, 'product-card__name')
precios = driver.find_elements(By.CLASS_NAME, 'product-card__price')

In [365]:
lista = []
for e in nombres:
    lista.append(e.text)

In [369]:
lista[18:24]

['Everlasting Hyperlight Transfer-Proof Liquid Lipstick',
 'Epic Kiss Nourishing Vegan Butter Lipstick',
 'Mini Epic Kiss Nourishing Vegan Butter Lipstick',
 'Studded Kiss Crème Lipstick',
 'XO Vinyl Lip Cream Lip Gloss',
 'XO Lip Gloss']

In [373]:
def extraer(URL):
    driver = webdriver.Chrome(PATH)
    driver.get(URL)
    
    driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()
    time.sleep(5)
    driver.find_element(By.CLASS_NAME, 'glClose').click()
    time.sleep(5)
    driver.execute_script('window.scrollTo(0, 2000);')  # 200px de scroll
    
    
    nombre = driver.find_elements(By.CLASS_NAME, 'product-card__name')
    precio = driver.find_elements(By.CLASS_NAME, 'product-card__price')
    link =  driver.find_elements(By.PARTIAL_LINK_TEXT, 'Lip')
    
    tmp = []
    precios = []
    links = []
    

    for e in nombre:
        tmp.append(e.text.split('\n'))
    
    for e in precio:
        precios.append(e.text)
    
    for e in link:
        links.append(e.get_attribute('href'))
    
    tmp = tmp[18:24]
    links = links[0:6]
    
    
    cols = ['NOMBRE']
    
    total = pd.DataFrame(tmp, columns = cols)

    
    total['PRECIO'] = precios
    total['LINKS'] = links
    total['PRODUCTO'] = 'SOMBRAS DE OJOS'
    
    
    driver.quit()
     
    return total

In [374]:
URL = 'https://kvdveganbeauty.com/collections/lip'

In [375]:
pintalabios_kvd = extraer(URL) #falta añadir las paletas

2022-12-10 19:56:23,397 [41429] WARNING  py.warnings:109: [JupyterRequire] /var/folders/fg/hm_rz3k53fv5qf5dywc0bfwr0000gn/T/ipykernel_41429/650098388.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH)

